In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import os
    
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

from itertools import cycle
from sklearn.mixture import GaussianMixture

In [34]:
ejem_encuesta_entrada=pd.read_excel('./datos/exa y encuestas/Examenes y encuestas 2022-20/Sección 01/Encuesta de entrada - Intentos individuales.xlsx')
ejem_encuesta_entrada

,Section #,Q #,Q Type,Q Title,Q Text,Bonus?,Difficulty,Answer,Answer Match,# Responses
0,Juan Felipe Blanco Talero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,Prueba de entrada,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,EM,NaN,Carrera,NaN,NaN,Ingeniería Civil,Sin seleccionar,0.0
3,NaN,1.0,EM,NaN,Carrera,NaN,NaN,Ingeniería Mecánica,Sin seleccionar,0.0
4,NaN,1.0,EM,NaN,Carrera,NaN,NaN,Ingeniería Electrónica,Sin seleccionar,0.0
...,...,...,...,...,...,...,...,...,...,...
1689,NaN,9.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,Python,Sin seleccionar,0.0
1690,NaN,9.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,Arduino,Sin seleccionar,0.0
1691,NaN,9.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,PHP,Sin seleccionar,0.0
1692,NaN,9.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,HTML,Sin seleccionar,0.0


In [35]:
ejem_prueba_entrada=pd.read_excel('./datos/exa y encuestas/Examenes y encuestas 2020-20/Seccion 03/Prueba de entrada - Intentos individuales.xlsx')
ejem_prueba_entrada

,Section #,Q #,Q Type,Q Title,Q Text,Bonus?,Difficulty,Answer,Answer Match,# Responses
0,Jacobo De La Barrera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,Prueba de entrada,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,RE,NaN,Carrera,NaN,NaN,Ingeniería Industrial,NaN,1.0
3,NaN,2.0,RE,NaN,Semestre,NaN,NaN,primer semestre,NaN,1.0
4,NaN,3.0,RE,NaN,¿Ha programado antes?,NaN,NaN,Si,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...
275,NaN,7.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,Python,Sin seleccionar,0.0
276,NaN,7.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,Arduino,Sin seleccionar,0.0
277,NaN,7.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,PHP,Sin seleccionar,0.0
278,NaN,7.0,SM,Tecnologías trabajadas,Marque las tecnologías con las que ha trabajad...,NaN,NaN,HTML,Sin seleccionar,0.0


In [58]:
def convertir_encuesta_entrada(df_encuesta,semestre,seccion):
    nuevas_columnas= ['SemestreAct','Estudiante','Seccion','Carrera','SemestreEst','ProgramoAntes','NivelIngles','CursoProgramacionU','VioAPO1','VioAPO2','VioHerr_Comp','VioTallerPython','VioTallerR','VioIP','VioSAD','VioOtro','ProgramoEnColegio',
                      'TrabajoConJava','TrabajoConCMM','TrabajoConC','TrabajoConVB','TrabajoConJavascript','TrabajoConPython','TrabajoConArduino','TrabajoConPHP','TrabajoConHTML','TrabajoConOtro']
    nuevos_datos=[]
    for i in range(0,len(df_encuesta),100):
        datos_act=[]
        
        termina= True
        if len(str(df_encuesta.iloc[i][0]))<=2:
            
            i_dif2= i-1
            
            def_temp = (df_encuesta.iloc[i_dif2:i_dif2+102]).copy()
            datos_act.append(semestre)
            #print(df_encuesta.iloc[i_dif2][0])
            #Nombre
            datos_act.append(df_encuesta.iloc[i_dif2][0])
            #Seccion
            datos_act.append(seccion)
            
        elif isinstance(df_encuesta.iloc[i][0], float):
            if math.isnan(df_encuesta.iloc[i][0]):
                i_dif1=i
                for j in range(100):
                    try:
                        if isinstance(df_encuesta.iloc[i_dif1+j][0], float):
                            continue
                        if(len(df_encuesta.iloc[i_dif1+j][0])>2 ):
                            datos_act.append(semestre)
                            #Nombre
                            #print(df_encuesta.iloc[i_dif1+j][0])
                            datos_act.append(df_encuesta.iloc[i_dif1+j][0])
                            i_dif1= i_dif1+j
                            termina=False
                            break
                    except:
                        break
                    
                if termina:
                    break
                ##Continuamos sacando informacion
                def_temp = (df_encuesta.iloc[i_dif1:i_dif1+102]).copy()
                #Seccion
                datos_act.append(seccion)
        else:
            
            def_temp = (df_encuesta.iloc[i:i+102]).copy()
            datos_act.append(semestre)
            #Nombre
            #print(df_encuesta.iloc[i][0])
            datos_act.append(df_encuesta.iloc[i][0])
            #Seccion
            datos_act.append(seccion)
        #Carrera
        if len(def_temp[(def_temp['Q #'] == 1.0) & (def_temp['# Responses'] != 0.0)])==0:
            datos_act.append("No responde")
        else:  
            datos_act.append((def_temp[(def_temp['Q #'] == 1.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer'])
        #Semestre
        if len(def_temp[(def_temp['Q #'] == 3.0) & (def_temp['# Responses'] != 0.0)])==0:
            datos_act.append("No responde")
        else:  
            datos_act.append((def_temp[(def_temp['Q #'] == 3.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer'])
        #ProgramoAntes
        if len(def_temp[(def_temp['Q #'] == 4.0) & (def_temp['# Responses'] != 0.0)])==0:
            datos_act.append("No responde")
        else:  
            datos_act.append((def_temp[(def_temp['Q #'] == 4.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer'])
        #NivelIngles
        if len(def_temp[(def_temp['Q #'] == 5.0) & (def_temp['# Responses'] != 0.0)])==0:
            datos_act.append("No responde")
        else:  
            datos_act.append((def_temp[(def_temp['Q #'] == 5.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer Match'])
        #CursoProgramacionU
        if len(def_temp[(def_temp['Q #'] == 6.0) & (def_temp['# Responses'] != 0.0)])==0:
            datos_act.append("No responde")
        else:  
            datos_act.append((def_temp[(def_temp['Q #'] == 6.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer'])
        #CursosU
        temp_temp= def_temp[(def_temp['Q #'] == 7.0)].copy()
        for k in range(len(temp_temp)):
            datos_act.append(temp_temp.iloc[k]['# Responses'])
        #ProgramoEnColegio
        if len(def_temp[(def_temp['Q #'] == 8.0) & (def_temp['# Responses'] != 0.0)])==0:
            datos_act.append("No responde")
        else:  
            datos_act.append((def_temp[(def_temp['Q #'] == 8.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer'])
        #Tecnología
        temp_temp= def_temp[(def_temp['Q #'] == 9.0)].copy()
        for k in range(len(temp_temp)):
            datos_act.append(temp_temp.iloc[k]['# Responses'])
#             print(datos_act)
        nuevos_datos.append(datos_act)
            
#     print(nuevos_datos)
    df_return = pd.DataFrame(nuevos_datos, columns=nuevas_columnas)
    return df_return
            

In [59]:
#convertir_encuesta_entrada(ejem_encuesta_entrada,'2021-20')

In [60]:
def convertir_prueba_entrada(df_prueba,semestre,seccion):
    nuevas_columnas= ['SemestreAct','Estudiante','Seccion','Carrera','SemestreEst','ProgramoAntes','NivelIngles','CursoProgramacionU','VioAPO1','VioAPO2','VioHerr_Comp','VioTallerPython','VioTallerR','VioIP','VioSAD','VioOtro','ProgramoEnColegio',
                      'TrabajoConJava','TrabajoConCMM','TrabajoConC','TrabajoConVB','TrabajoConJavascript','TrabajoConPython','TrabajoConArduino','TrabajoConPHP','TrabajoConHTML','TrabajoConOtro']
    nuevos_datos=[]
    for i in range(0,len(df_prueba),25):
#         print(i)
        datos_act=[]
        termina= True
        if isinstance(df_prueba.iloc[i][0], float):
            if math.isnan(df_prueba.iloc[i][0]):
                i_dif1=i
                for j in range(25):
                    try:
                        if isinstance(df_prueba.iloc[i_dif1+j][0], float):
                            continue
                        if(len(df_prueba.iloc[i_dif1+j][0])>2 ):
                            datos_act.append(semestre)
                            #Nombre
                            #print(df_prueba.iloc[i_dif1+j][0])
                            datos_act.append(df_prueba.iloc[i_dif1+j][0])
                            i_dif1= i_dif1+j
                            termina=False
                            break    
                    except:
                        break
                if termina:
                    break
                #
                def_temp = (df_prueba.iloc[i_dif1:i_dif1+27]).copy()
                #Seccion
                datos_act.append(seccion)
                
        elif len(df_prueba.iloc[i][0])<=2 :
            i=i-1
            
            def_temp = (df_prueba.iloc[i:i+27]).copy()
            datos_act.append(semestre)
            #print((df_prueba.iloc[i][0]))
            datos_act.append(df_prueba.iloc[i][0])
            #Seccion
            datos_act.append(seccion)
        else:
        
            def_temp = (df_prueba.iloc[i:i+27]).copy()
            datos_act.append(semestre)
            #Nombre
            #print(df_prueba.iloc[i][0])
            datos_act.append(df_prueba.iloc[i][0])
            #Seccion
            datos_act.append(seccion)
        #Carrera
        datos_act.append((def_temp[(def_temp['Q #'] == 1.0)]).iloc[0]['Answer'])
        #Semestre
        if len((def_temp[(def_temp['Q #'] == 2.0)])) == 0:
                datos_act.append("-1")
        else:
            datos_act.append((def_temp[(def_temp['Q #'] == 2.0)]).iloc[0]['Answer'])
        #ProgramoAntes
        datos_act.append((def_temp[(def_temp['Q #'] == 3.0)]).iloc[0]['Answer'])
        #NivelIngles
        datos_act.append((def_temp[(def_temp['Q #'] == 4.0) & (def_temp['# Responses'] != 0.0)]).iloc[0]['Answer Match'])
        #CursoProgramacionU
        if len((def_temp[(def_temp['Q #'] == 5.0)])) == 0:
                datos_act.append("No responde")
        else:
            datos_act.append((def_temp[(def_temp['Q #'] == 5.0)]).iloc[0]['Answer'])
        #CursosU
        for k in range(8):
            datos_act.append(-1)
        #ProgramoEnColegio
        datos_act.append((def_temp[(def_temp['Q #'] == 6.0)]).iloc[0]['Answer'])
        #TrabajoCon
        temp_temp= def_temp[(def_temp['Q #'] == 7.0)].copy()
        for k in range(len(temp_temp)):
            datos_act.append(temp_temp.iloc[k]['# Responses'])
        nuevos_datos.append(datos_act)
        
#     print(nuevos_datos)
    df_return = pd.DataFrame(nuevos_datos, columns=nuevas_columnas)
    return df_return

In [61]:
#convertir_prueba_entrada(ejem_prueba_entrada,'2020-20')

# Conversion archivos Encuestas y Pruebas a Archivo Unificado

In [40]:
path = os.getcwd()
files = os.listdir('./datos/exa y encuestas/')
lista_examen = []
lista_encuesta = []
lista_prueba = []
lista_otros = []

for root, dirs, files in os.walk("./datos/exa y encuestas/"):
    
    for file in files:
        if file.endswith(".xlsx"):
            if file.startswith("Examen"):
                lista_examen.append(os.path.join(root, file))
            elif file.startswith("Encuesta"):
                lista_encuesta.append(os.path.join(root, file))
            elif file.startswith("Prueba"):
                lista_prueba.append(os.path.join(root, file))
            else:
                lista_otros.append(os.path.join(root, file))

print(len(lista_encuesta))
print(len(lista_examen))
print(len(lista_prueba))
print(len(lista_otros))

174
217
46
2


In [62]:
nuevas_columnas= ['SemestreAct','Estudiante','Seccion','Carrera','SemestreEst','ProgramoAntes','NivelIngles','CursoProgramacionU','VioAPO1','VioAPO2','VioHerr_Comp','VioTallerPython','VioTallerR','VioIP','VioSAD','VioOtro','ProgramoEnColegio',
                      'TrabajoConJava','TrabajoConCMM','TrabajoConC','TrabajoConVB','TrabajoConJavascript','TrabajoConPython','TrabajoConArduino','TrabajoConPHP','TrabajoConHTML','TrabajoConOtro']
df_pruebas_final = pd.DataFrame(columns=nuevas_columnas)
for link in lista_prueba:
    semestre= link.split("\\")[0].split("/")[3].replace("Examenes y encuestas ","").replace("-","")
    seccion= link.split("\\")[1].replace("Seccion ","").replace("-","")
    if seccion[0]=="0":
        seccion=seccion.replace("0","")
    df_archivo=pd.read_excel(link)
    print(link)
    df_actual=convertir_prueba_entrada(df_archivo,semestre,seccion)
    df_pruebas_final = pd.concat([df_pruebas_final,df_actual],ignore_index=True)
df_pruebas_final
    

./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 01\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 02\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 03\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 04\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 05\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 06\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 07\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 08\Prueba de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2020-20\Seccion 09\Prueba de entrada - Intentos individuales.xlsx
.

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,CursoProgramacionU,VioAPO1,VioAPO2,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
0,202020,Nicolas Felipe Alonso Zea,1,Ing. Civil,4to,Si,7,"Introducción a la Programación (ISIS 1221), el...",-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,202020,Juan Camilo Colmenares Ortiz,1,Ingeniería mecánica,Primer semestre,"Si, He programado en Android Studio con Java y...",8,No he tomado ningún curso de programación en l...,-1,-1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,202020,Nicole Cundy Pombo,1,Ingeniería Industrial,1,No,8,No,-1,-1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202020,Sebastian Alberto De Mendoza Corrales,1,microbiologia,primer semestre,no,6,no,-1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,202020,Sofia Ossa Rubio,1,Ingenieria Industrial,Primero,No,7,No,-1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,202020,Cristian Santiago Marin Mayorga,47,Ing biomedica,3,No,5,asistente a unas clases de ip,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
582,202020,Andres Leonardo Ospina Rios,47,ingeniería biomédica,segundo semestre,muy poco,3,no,-1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
583,202020,Sergio Andres Pinzon Jimenez,47,Ingenieria Biomedica,Segundo,No,8,No,-1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
584,202020,Juan Esteban Rodriguez Molina,47,ingenieria de sistemas,segundo,si,6,no,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [63]:
nuevas_columnas= ['SemestreAct','Estudiante','Seccion','Carrera','SemestreEst','ProgramoAntes','NivelIngles','CursoProgramacionU','VioAPO1','VioAPO2','VioHerr_Comp','VioTallerPython','VioTallerR','VioIP','VioSAD','VioOtro','ProgramoEnColegio',
                      'TrabajoConJava','TrabajoConCMM','TrabajoConC','TrabajoConVB','TrabajoConJavascript','TrabajoConPython','TrabajoConArduino','TrabajoConPHP','TrabajoConHTML','TrabajoConOtro']
df_encuestas_final = pd.DataFrame(columns=nuevas_columnas)
for link in lista_encuesta:
    semestre= link.split("\\")[0].split("/")[3].replace("Examenes y encuestas ","").replace("-","")
    seccion= link.split("\\")[1].replace("Seccion ","").replace("-","")
    if seccion[0]=="0":
        seccion=seccion.replace("0","")
    df_archivo=pd.read_excel(link)
    print(link)
    df_actual=convertir_encuesta_entrada(df_archivo,semestre,seccion)
    df_encuestas_final = pd.concat([df_encuestas_final,df_actual],ignore_index=True)
df_encuestas_final

./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 01\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 02\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 03\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 04\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 05\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 06\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 07\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 08\Encuesta de entrada - Intentos individuales.xlsx
./datos/exa y encuestas/Examenes y encuestas 2021-10\Seccion 09\Encuesta de entrada - Intentos i

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,CursoProgramacionU,VioAPO1,VioAPO2,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
0,202110,Juan Sebastian Arias,1,Ingeniería Química,1,Verdadero,8,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,202110,Laura Camila Avella Fonseca,1,Ingeniería Eléctrica,1,Falso,4,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,202110,Santiago Ballesteros Varon,1,Ingeniería Industrial,1,Falso,3,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202110,David Diaz Sanchez,1,Física,1,Falso,8,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,202110,Carol Sofia Florido Castro,1,Ingeniería Industrial,1,Falso,5,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,202220,Luna Sofia Ramos Gualtero,Sección 98,Ingeniería General,1,Falso,6,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2013,202220,Miguel Angel Rodriguez Huertas,Sección 98,Ing. De Sistemas Y Computación,8,Verdadero,6,Falso,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2014,202220,Jose Abrahan Briceño Figueroa,Sección 99,Ing. De Sistemas Y Computación,1,Falso,3,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,202220,Dilan Estiven Caro Palencia,Sección 99,Ing. De Sistemas Y Computación,1,Verdadero,3,Falso,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [64]:
df_casi_final = pd.concat([df_encuestas_final,df_pruebas_final],ignore_index=True)
df_casi_final

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,CursoProgramacionU,VioAPO1,VioAPO2,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
0,202110,Juan Sebastian Arias,1,Ingeniería Química,1,Verdadero,8,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,202110,Laura Camila Avella Fonseca,1,Ingeniería Eléctrica,1,Falso,4,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,202110,Santiago Ballesteros Varon,1,Ingeniería Industrial,1,Falso,3,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202110,David Diaz Sanchez,1,Física,1,Falso,8,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,202110,Carol Sofia Florido Castro,1,Ingeniería Industrial,1,Falso,5,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,202020,Cristian Santiago Marin Mayorga,47,Ing biomedica,3,No,5,asistente a unas clases de ip,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2599,202020,Andres Leonardo Ospina Rios,47,ingeniería biomédica,segundo semestre,muy poco,3,no,-1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2600,202020,Sergio Andres Pinzon Jimenez,47,Ingenieria Biomedica,Segundo,No,8,No,-1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2601,202020,Juan Esteban Rodriguez Molina,47,ingenieria de sistemas,segundo,si,6,no,-1,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# Conversión Archivo Unificado 2020-1

In [65]:
unificado_202010=pd.read_excel('./datos/exa y encuestas/Resultados Encuestas 2020-1.xlsx',sheet_name="Tabulación")
unificado_202010= unificado_202010[['SECC','NOMBRE CURSO','CARNET','APELLIDOS','NOMBRES','PROGRAMA ESTUD','DOBLE PROG','EMAIL','Semestre','Lenguajes','Cursos','Colegio','Ingles']]

In [66]:
unificado_202010

,SECC,NOMBRE CURSO,CARNET,APELLIDOS,NOMBRES,PROGRAMA ESTUD,DOBLE PROG,EMAIL,Semestre,Lenguajes,Cursos,Colegio,Ingles
0,1,INTRODUCCIÓN A LA PROGRAMACIÓN,201813809,HEREDIA NIÑO,JUAN DIEGO,ECONOMIA,ADMI,jd.heredian@uniandes.edu.co,5,Arduino,no,si,5.0
1,1,INTRODUCCIÓN A LA PROGRAMACIÓN,201911264,ALARCON PERICO,MARIA PAULA,INGENIERIA ELECTRONICA,NaN,mp.alarcon@uniandes.edu.co,1,Arduino,no,si,5.0
2,1,INTRODUCCIÓN A LA PROGRAMACIÓN,201913232,MEJIA CARDONA,JOSE MIGUEL,INGENIERIA INDUSTRIAL,NaN,jm.mejiac@uniandes.edu.co,1,no,no,no,5.0
3,1,INTRODUCCIÓN A LA PROGRAMACIÓN,201920787,CAÑON CAÑON,MAICOL DAVID,INGENIERIA INDUSTRIAL,NaN,m.canon@uniandes.edu.co,1,no,no,no,5.0
4,1,INTRODUCCIÓN A LA PROGRAMACIÓN,201922295,GARCIA CARVAJAL,LORENA,INGENIERIA INDUSTRIAL,NaN,l.garcia1@uniandes.edu.co,1,HTML,Campamento la tecnologia mas cerca a ti,si,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,40,INTRODUCCIÓN A LA PROGRAMACIÓN,202011753,CASTELLANOS PABON,LAYLA AKEMI,MICROBIOLOGIA,NaN,la.castellanosp1@uniandes.edu.co,1,no,no,no,5.0
912,40,INTRODUCCIÓN A LA PROGRAMACIÓN,202012274,GONZALEZ MONTEALEGRE,SANTIAGO,INGENIERIA INDUSTRIAL,NaN,s.gonzalez35@uniandes.edu.co,1,"c++,visual basic, html",no,si,5.0
913,40,INTRODUCCIÓN A LA PROGRAMACIÓN,202013493,RUEDA DIAZ,JOSE DAVID,INGENIERIA ELECTRONICA,NaN,jd.ruedad1@uniandes.edu.co,1,no,no,no,3.0
914,40,INTRODUCCIÓN A LA PROGRAMACIÓN,202015520,MANRIQUE BARRERA,STEFANNY ALEXANDRA,INGENIERIA QUIMICA,NaN,s.manriqueb@uniandes.edu.co,1,no,no,si,5.0


In [67]:
def convertir_excel_202010(df_encuestas):
    nuevas_columnas= ['SemestreAct','Estudiante','Seccion','Carrera','SemestreEst','ProgramoAntes','NivelIngles','CursoProgramacionU','VioAPO1','VioAPO2','VioHerr_Comp','VioTallerPython','VioTallerR','VioIP','VioSAD','VioOtro','ProgramoEnColegio',
                      'TrabajoConJava','TrabajoConCMM','TrabajoConC','TrabajoConVB','TrabajoConJavascript','TrabajoConPython','TrabajoConArduino','TrabajoConPHP','TrabajoConHTML','TrabajoConOtro']
    datos_nuevos=[]
    for i in range(len(df_encuestas)):
        datos_act=[]
        #Semestre
        datos_act.append('202010')
        #Nombre
        datos_act.append(df_encuestas.iloc[i]['NOMBRES']+" "+df_encuestas.iloc[i]['APELLIDOS'])
        #Seccion
        datos_act.append(df_encuestas.iloc[i]['SECC'])
        #Carrera
        datos_act.append(df_encuestas.iloc[i]['PROGRAMA ESTUD'])
        #SemestreEst
        datos_act.append(df_encuestas.iloc[i]['Semestre'])
        #Programo antes
        if df_encuestas.iloc[i]['Lenguajes'] == "no" and df_encuestas.iloc[i]['Cursos'] == "no":
            datos_act.append('Falso')
        else:
            datos_act.append('Verdadero')
        #Nivel Ingles
        datos_act.append(df_encuestas.iloc[i]['Ingles'])
        #CursoProgramacionU y Vio Materias
        if df_encuestas.iloc[i]['Cursos'] == "no":
            datos_act.append('Falso')
        else:
            datos_act.append('Verdadero')
#         print(df_encuestas.iloc[i]['Cursos'])
        if isinstance(df_encuestas.iloc[i]['Cursos'], float):
#             print("Entro")
            for k in range(8):
                datos_act.append(0)
        elif (len(df_encuestas.iloc[i]['Cursos'].split(",")) >= 1) and df_encuestas.iloc[i]['Cursos'].split(",")[0] != "no":
            apo1=0
            apo2=0
            herr=0
            tallerpy=0
            tallerR=0
            ip=0
            sad=0
            otro=0
            materiasU=df_encuestas.iloc[i]['Cursos'].split(",")
            for k in materiasU:
                act=k.lower().replace(" ","")
                
                if ("apo1" == act):
                    apo1=1
                elif ("apo2" == act):
                    apo2=1
                elif ("ip" == act):
                    ip=1
                elif ("sad" == act):
                    sad=1
                else:
                    otro=1
            datos_act.append(apo1)
            datos_act.append(apo2)
            datos_act.append(herr)
            datos_act.append(tallerpy)
            datos_act.append(tallerR)
            datos_act.append(ip)
            datos_act.append(sad)
            datos_act.append(otro)
        else:
            for k in range(8):
                datos_act.append(0)
        #Programo en Colegio
        datos_act.append(df_encuestas.iloc[i]['Colegio'])
        #Tecnologías
        if isinstance(df_encuestas.iloc[i]['Lenguajes'], float):
#             print("Entro")
            for k in range(8):
                datos_act.append(0)
        elif len(df_encuestas.iloc[i]['Lenguajes'].split(",")) >= 1 and df_encuestas.iloc[i]['Lenguajes'].split(",")[0] != "no":
            java=0
            cmm=0
            c=0
            visual_basic=0
            js=0
            python=0
            arduino=0
            php=0
            html=0
            otro=0
            tecnologias= df_encuestas.iloc[i]['Lenguajes'].split(",")
            for k in tecnologias:
                act=k.lower().replace(" ","")
                
                if ("java" == act):
                    java=1
                elif ("c++" == act):
                    cmm=1
                elif ("c" == act):
                    c=1
                elif ("visualbasic" == act):
                    visual_basic=1
                elif ("javascript" == act):
                    js=1
                elif ("python" == act):
                    python=1
                elif ("arduino" == act):
                    arduino=1 
                elif ("php" == act):
                    php=1
                elif ("html" == act):
                    html=1
                else:
                    otro=1
            datos_act.append(java)
            datos_act.append(cmm)
            datos_act.append(c)
            datos_act.append(visual_basic)
            datos_act.append(js)
            datos_act.append(python)
            datos_act.append(arduino)
            datos_act.append(php)
            datos_act.append(html)
            datos_act.append(otro)
        else:
            for k in range(10):
                datos_act.append(0)
        datos_nuevos.append(datos_act)
    df_return = pd.DataFrame(datos_nuevos, columns=nuevas_columnas)
    return df_return      
            

In [68]:
df_202010=convertir_excel_202010(unificado_202010)

# Dataset Final

In [69]:
df_final = pd.concat([df_casi_final,df_202010],ignore_index=True)
df_final

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,CursoProgramacionU,VioAPO1,VioAPO2,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
0,202110,Juan Sebastian Arias,1,Ingeniería Química,1,Verdadero,8,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,202110,Laura Camila Avella Fonseca,1,Ingeniería Eléctrica,1,Falso,4,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,202110,Santiago Ballesteros Varon,1,Ingeniería Industrial,1,Falso,3,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202110,David Diaz Sanchez,1,Física,1,Falso,8,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,202110,Carol Sofia Florido Castro,1,Ingeniería Industrial,1,Falso,5,Falso,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,202010,LAYLA AKEMI CASTELLANOS PABON,40,MICROBIOLOGIA,1,Falso,5.0,Falso,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3515,202010,SANTIAGO GONZALEZ MONTEALEGRE,40,INGENIERIA INDUSTRIAL,1,Verdadero,5.0,Falso,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3516,202010,JOSE DAVID RUEDA DIAZ,40,INGENIERIA ELECTRONICA,1,Falso,3.0,Falso,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3517,202010,STEFANNY ALEXANDRA MANRIQUE BARRERA,40,INGENIERIA QUIMICA,1,Falso,5.0,Falso,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
df_final.to_csv("./datos/todo_encuestas.csv",index=False,sep=";")
df_final.to_excel("./datos/todo_encuestas.xlsx",index=False)

# Modificaciones datos encuesta

In [104]:
encuestas= pd.read_excel('./datos/todo_encuestas.xlsx')
encuestas= encuestas.dropna()
encuestas

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,CursoProgramacionU,VioAPO1,VioAPO2,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
0,202110,Juan Sebastian Arias,1,Ingeniería Química,1,Verdadero,8,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,202110,Laura Camila Avella Fonseca,1,Ingeniería Eléctrica,1,Falso,4,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,202110,Santiago Ballesteros Varon,1,Ingeniería Industrial,1,Falso,3,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202110,David Diaz Sanchez,1,Física,1,Falso,8,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,202110,Carol Sofia Florido Castro,1,Ingeniería Industrial,1,Falso,5,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,202010,LAYLA AKEMI CASTELLANOS PABON,40,MICROBIOLOGIA,1,Falso,5,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3515,202010,SANTIAGO GONZALEZ MONTEALEGRE,40,INGENIERIA INDUSTRIAL,1,Verdadero,5,Falso,0,0,...,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3516,202010,JOSE DAVID RUEDA DIAZ,40,INGENIERIA ELECTRONICA,1,Falso,3,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3517,202010,STEFANNY ALEXANDRA MANRIQUE BARRERA,40,INGENIERIA QUIMICA,1,Falso,5,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s


In [106]:
def programo_antes_proceso_de_texto(programoantes):
    programoantes= normalize(programoantes.lower())
    if (programoantes == "falso") or (programoantes.startswith("no")) or (programoantes.startswith("nunca")) or (programoantes.startswith("muy poco")) or ("nada" in programoantes) or (programoantes.startswith("jamas")) or (programoantes.startswith("vagamente")):
        return "Falso"
    else:
        return "Verdadero"


In [107]:
encuestas['ProgramoAntes'] = encuestas['ProgramoAntes'].apply(programo_antes_proceso_de_texto)

In [108]:
# encuestas.groupby(['ProgramoAntes']).count()

In [109]:
lista_carreras=[
"ingenieria civil",
"ingenieria mecanica",
"ingeniería electrónica",
"ingeniería quimica",
"ing de sistemas y computación",
"matematicas",
"Ingeniería Eléctrica",
"Ingeniería Industrial",
"Ingeniería General",
"Administración De Empresas",
"Física",
"Música",
"Ingeniería Ambiental",
"Ciencia Política",
"Química",
"Medicina",
"Economía",
"Arte",
"Estudios Dirigidos",
"Arquitectura",
"Biología",
"Microbiología",
"Filosofía",
"Diseño",
"Literatura",
"Derecho",
"Lenguajes Y Estudios Socioculturales",
"Geociencias",
"Historia",
"Ingeniería Biomédica",
"Educación Continuada",
"Antropología",
"Psicología",
"Extensión",
"Historia Del Arte",
"Contaduría Internacional",
"Maestr.Biología Computacional",
"Lenguas Y Cultura",
"Licenciatura En Matemáticas",
"Maestría En Cien.Biol.Area Mic",
"Licenciat.Ciencias Naturales",
"Gobierno Y Asuntos Públicos",
"Estudios Gener.De Intercambio",
"Programa Básico Ingeniería",
"Estud.Gener.Intercamb.Maestría",
"Licenciatura En Física",
"Extensión Universitaria",
"ingenieria de alimentos",
"narrativas digitales",
]
def carrera_proceso_de_texto(carrera):
    carrera = normalize(carrera.lower()).replace(".","")
    for i in lista_carreras:
        act= normalize(i.lower()).replace(".","")
        retorno=None
        if act in carrera:
            retorno=act
            break
        elif act.replace("ingenieria","ing") in carrera:
            retorno=act
            break
            
    if retorno == None:
        if "sistemas" in carrera:
            retorno= "ing de sistemas y computacion"
        elif "administracion" in carrera:
            retorno= "administracion de empresas"
        else:
            retorno= "Otra"
    return retorno
        


In [110]:
encuestas['Carrera']=encuestas['Carrera'].apply(carrera_proceso_de_texto)

In [111]:
def semestre_proceso_de_texto(semestre):
    if isinstance(semestre, int):
        return semestre
    else:
        semestre = normalize(semestre.lower()).replace(".","")
        retorno=-1
        if ("primer" in semestre) or ("1" in semestre):
            retorno= 1
        elif ("segundo" in semestre) or ("segudon" in semestre) or ("2" in semestre):
            retorno= 2
        elif ("tercer" in semestre) or ("3" in semestre):
            retorno= 3
        elif ("cuarto" in semestre) or ("4" in semestre):
            retorno= 4
        elif ("quinto" in semestre) or ("5" in semestre):
            retorno= 5
        elif ("sexto" in semestre) or ("6" in semestre):
            retorno= 6
        elif ("septimo" in semestre) or ("7" in semestre):
            retorno= 7
        elif ("octavo" in semestre) or ("8" in semestre):
            retorno= 8
        elif ("noveno" in semestre) or ("9" in semestre):
            retorno= 9
        elif ("decimo" in semestre) or ("10" in semestre):
            retorno= 10
        elif ("11" in semestre):
            retorno= 11
        elif ("12" in semestre):
            retorno= 12
        return retorno

In [112]:
encuestas['SemestreEst'] = encuestas['SemestreEst'].apply(semestre_proceso_de_texto)

In [113]:
def cursoProgramacion_proceso_de_texto(cursoProgramacionU):
    cursoProgramacionU = normalize(cursoProgramacionU.lower())
    retorno = ""
    if "no" in cursoProgramacionU or "falso" in cursoProgramacionU or "nunca" in cursoProgramacionU or "muy poco" in cursoProgramacionU or "nada" in cursoProgramacionU or "jamas" in cursoProgramacionU or "ninguno" in cursoProgramacionU:
        retorno = "Falso"
    elif "si" in cursoProgramacionU or "verdadero" in cursoProgramacionU:
        retorno = "Verdadero"
    else:
        retorno = "Verdadero"
    return retorno

In [114]:
encuestas['CursoProgramacionU'] = encuestas['CursoProgramacionU'].apply(cursoProgramacion_proceso_de_texto)

In [115]:
encuestas.groupby(['CursoProgramacionU']).count()

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,VioAPO1,VioAPO2,VioHerr_Comp,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
CursoProgramacionU,,,,,,,,,,,,,,,,,,,,,
Falso,2809,2809,2809,2809,2809,2809,2809,2809,2809,2809,...,2809,2809,2809,2809,2809,2809,2809,2809,2809,2809
Verdadero,618,618,618,618,618,618,618,618,618,618,...,618,618,618,618,618,618,618,618,618,618


In [116]:
def programa_colegio_proceso_de_texto(programoenColegio):
    programoenColegio = normalize(programoenColegio.lower())
    retorno= "Verdadero"
    if "no" in programoenColegio or "falso" in programoenColegio or "nunca" in programoenColegio or "muy poco" in programoenColegio or "nada" in programoenColegio or "jamas" in programoenColegio or "ninguno" in programoenColegio:
        retorno = "Falso"
    else:
        retorno= "Verdadero"
    return retorno

In [117]:
encuestas['ProgramoEnColegio'] = encuestas['ProgramoEnColegio'].apply(programa_colegio_proceso_de_texto)

In [118]:
# encuestas.groupby(['ProgramoEnColegio']).count()

In [119]:
encuestas

,SemestreAct,Estudiante,Seccion,Carrera,SemestreEst,ProgramoAntes,NivelIngles,CursoProgramacionU,VioAPO1,VioAPO2,...,TrabajoConJava,TrabajoConCMM,TrabajoConC,TrabajoConVB,TrabajoConJavascript,TrabajoConPython,TrabajoConArduino,TrabajoConPHP,TrabajoConHTML,TrabajoConOtro
0,202110,Juan Sebastian Arias,1,ingenieria quimica,1,Verdadero,8,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,202110,Laura Camila Avella Fonseca,1,ingenieria electrica,1,Falso,4,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,202110,Santiago Ballesteros Varon,1,ingenieria industrial,1,Falso,3,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202110,David Diaz Sanchez,1,fisica,1,Falso,8,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,202110,Carol Sofia Florido Castro,1,ingenieria industrial,1,Falso,5,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,202010,LAYLA AKEMI CASTELLANOS PABON,40,biologia,1,Falso,5,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3515,202010,SANTIAGO GONZALEZ MONTEALEGRE,40,ingenieria industrial,1,Verdadero,5,Falso,0,0,...,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3516,202010,JOSE DAVID RUEDA DIAZ,40,ingenieria electronica,1,Falso,3,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3517,202010,STEFANNY ALEXANDRA MANRIQUE BARRERA,40,ingenieria quimica,1,Falso,5,Falso,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
encuestas.to_excel("./datos/todo_encuestas2.xlsx",index=False)